# Ver1. origin user 이용

## sparsity 고려하지 않고 origin rating data를 이용해 LightFM 모델을 학습해봅니다.
---
### 1. rating 결측치 처리
### 2. 정규화 처리 여부에 따른 비교

In [1]:
import os
import pandas as pd
import numpy as  np

In [2]:
os.getcwd()

'C:\\Users\\SSAFY\\Desktop\\Whisky\\v1'

In [3]:
rating = pd.read_csv("../dataset/rating.csv", index_col = 0, encoding="utf-8")
whisky = pd.read_csv("../dataset/whisky.csv", index_col=0 , encoding="UTF-8")
whisky_index = pd.read_csv("../dataset/whisky_index.csv", index_col=0 , encoding="UTF-8")
user_index = pd.read_csv("../dataset/user_index.csv", index_col=0 , encoding="UTF-8")

## rating 결측치 처리 - 평균으로 대체합니다.
---
### 1. interactions 결측치 처리
### 2. whisky columns(avr_rating) 수정

## 1. interactions 결측치 처리

In [4]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 908176 entries, 0 to 11258
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    908176 non-null  int64  
 1   whisky_id  908176 non-null  int64  
 2   rating     888132 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 27.7 MB


In [5]:
# Reset the index of the DataFrame
rating = rating.reset_index(drop=True)

# Group by "whisky_name" and fill missing values in "rating" with the rounded mean of that group
rating["rating"] = rating.groupby("whisky_id")["rating"].apply(lambda x: x.fillna(round(x.mean()) if not np.isnan(x.mean()) else 0))

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_12228\1581247862.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  rating["rating"] = rating.groupby("whisky_id")["rating"].apply(lambda x: x.fillna(round(x.mean()) if not np.isnan(x.mean()) else 0))


In [6]:
rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908176 entries, 0 to 908175
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    908176 non-null  int64  
 1   whisky_id  908176 non-null  int64  
 2   rating     908176 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 20.8 MB


## 2. whisky 평균 평점, 총 인원 수정

In [7]:
whisky["avr_rating"] = round(rating.groupby("whisky_id")["rating"].mean(), 2)

In [8]:
whisky["total_rating"] = rating.groupby("whisky_id")['user_id'].count()

In [9]:
whisky

,whisky_id,link,image,name,avr_rating,category,location,total_rating,cost_rank,abv,...,herbal,oily,full_bodied,rich,sweet,salty,vanilla,tart,fruity,floral
0,0,/spirits/hibiki-21-year,https://ip-distiller.imgix.net/images/spirits/...,Hibiki 21 Year,9.01,Blended,Japan,861.0,5,43.00,...,30,20,80,80,85,15,20,25,85,50
1,1,/spirits/highland-park-18,https://ip-distiller.imgix.net/images/spirits/...,Highland Park 18 Year,8.89,Peated Single Malt,"Islands, Scotland",2988.0,4,43.00,...,20,40,70,80,70,40,50,50,70,20
2,2,/spirits/michter-s-20-year-kentucky-straight-b...,https://ip-distiller.imgix.net/images/spirits/...,Michter's 20 Year Kentucky Straight Bourbon (2...,9.00,Bourbon,"Kentucky, USA",10.0,5,57.10,...,0,15,80,90,85,5,30,25,35,0
3,3,/spirits/george-t-stagg-bourbon-fall-2019,https://ip-distiller.imgix.net/images/spirits/...,George T. Stagg Bourbon (Fall 2019),9.06,Bourbon,"Kentucky, USA",629.0,4,58.45,...,50,20,60,60,45,0,60,60,45,0
4,4,/spirits/bowmore-mizunara-cask-finish,https://ip-distiller.imgix.net/images/spirits/...,Bowmore Mizunara Cask Finish,7.86,Peated Single Malt,"Islay, Scotland",22.0,5,53.90,...,30,10,75,75,60,20,30,20,50,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,3530,/spirits/wild-turkey-spiced,https://ip-distiller.imgix.net/images/spirits/...,Wild Turkey Spiced,5.92,Flavored Whiskey,"Kentucky, USA",13.0,1,43.00,...,0,0,40,55,85,0,25,10,10,0
3531,3531,/spirits/seagram-s-seven-crown-american-blende...,https://ip-distiller.imgix.net/images/spirits/...,Seagram's 7 Crown American Blended Whiskey,4.31,Blended American Whiskey,USA,237.0,1,40.00,...,20,0,30,0,100,0,100,0,60,10
3532,3532,/spirits/11-wells-single-malt-whiskey,https://ip-distiller.imgix.net/images/spirits/...,11 Wells Single Malt Whiskey,6.00,American Single Malt,"Minnesota, USA",2.0,3,42.00,...,10,40,10,10,90,0,50,20,30,10
3533,3533,/spirits/immortal-spirits-early-whiskey,https://ip-distiller.imgix.net/images/spirits/...,Immortal Spirits Early Whiskey,2.00,Other Whiskey,"Oregon, USA",1.0,2,44.50,...,100,80,0,0,40,0,20,0,0,0


### 결측치 평균으로 대체 완료

## 2-1. 정규화 진행하지 않고 학습 진행

In [10]:
import os

from scipy.sparse import coo_matrix, csr_matrix
from tqdm.notebook import tqdm

from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, auc_score, recall_at_k, reciprocal_rank


C:\Users\SSAFY\anaconda3\envs\mini-project\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


#### item meta data 생성

In [11]:
columns = whisky.columns.tolist()
columns = [columns[8]] + columns[11:]
columns

['cost_rank',
 'smoky',
 'peaty',
 'spicy',
 'herbal',
 'oily',
 'full_bodied',
 'rich',
 'sweet',
 'salty',
 'vanilla',
 'tart',
 'fruity',
 'floral']

In [12]:
item_features = np.zeros((whisky['whisky_id'].nunique(), len(columns)))
for i in range(whisky['whisky_id'].nunique()):
    for j, col_name in enumerate(columns):
        item_features[i, j] = whisky.iloc[i][col_name]
# item_features = csr_matrix(item_features)

In [13]:
item_features

array([[ 5., 30., 85., ..., 25., 85., 50.],
       [ 4., 40., 30., ..., 50., 70., 20.],
       [ 5., 15.,  0., ..., 25., 35.,  0.],
       ...,
       [ 3., 90., 10., ..., 20., 30., 10.],
       [ 2.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1., 31.,  0., ...,  0., 30., 10.]])

In [14]:
item_features = csr_matrix(item_features)

#### user_item_interactions 만들기

In [15]:
user_index['user_id'].nunique()

119515

In [16]:
whisky_index['whisky_id'].nunique()

3535

In [17]:
interactions = coo_matrix(
    (
        rating['rating'].values,
        (rating['user_id'].values, rating["whisky_id"].values)
    ),
    shape=(user_index['user_id'].nunique(), whisky_index['whisky_id'].nunique())
)

In [18]:
print(interactions)
interactions

  (0, 0)	9.0
  (1, 0)	7.0
  (2, 0)	9.0
  (3, 0)	10.0
  (4, 0)	9.0
  (5, 0)	7.0
  (6, 0)	10.0
  (7, 0)	7.0
  (8, 0)	10.0
  (9, 0)	10.0
  (10, 0)	9.0
  (11, 0)	9.0
  (12, 0)	8.0
  (13, 0)	8.0
  (14, 0)	10.0
  (15, 0)	10.0
  (16, 0)	9.0
  (17, 0)	9.0
  (18, 0)	8.0
  (19, 0)	9.0
  (20, 0)	9.0
  (21, 0)	10.0
  (22, 0)	9.0
  (23, 0)	9.0
  (24, 0)	9.0
  :	:
  (7636, 3531)	4.0
  (56292, 3531)	2.0
  (2670, 3531)	2.0
  (786, 3531)	2.0
  (5942, 3531)	6.0
  (15561, 3531)	2.0
  (25623, 3532)	6.0
  (2885, 3532)	6.0
  (7368, 3533)	2.0
  (69554, 3534)	10.0
  (4484, 3534)	2.0
  (85727, 3534)	5.0
  (43900, 3534)	4.0
  (16070, 3534)	8.0
  (29998, 3534)	8.0
  (7895, 3534)	4.0
  (104052, 3534)	5.0
  (31152, 3534)	2.0
  (119512, 3534)	6.0
  (2509, 3534)	4.0
  (4828, 3534)	4.0
  (119513, 3534)	2.0
  (119514, 3534)	8.0
  (3123, 3534)	4.0
  (95648, 3534)	2.0


<119515x3535 sparse matrix of type '<class 'numpy.float64'>'
	with 908176 stored elements in COOrdinate format>

#### Train_Test data split

In [19]:
from lightfm.cross_validation import random_train_test_split
from time import time

In [20]:
train_interactions, test_interactions = random_train_test_split(interactions, test_percentage=0.2, random_state=42)

### Hyper Parameter Optimization by using HyperOPT

In [ ]:
from hyperopt import fmin, hp, tpe, Trials

In [ ]:
trials = Trials()
space = [
    hp.choice('no_components', range(20,80,10)),
    hp.uniform('learning_rate', 0.01, 0.05),
    hp.uniform('item_alpha', 1e-05, 5e-05),
]

In [ ]:
def objective(params):
    no_components, learning_rate, item_alpha = params

    model = LightFM(no_components=no_components,
                    learning_schedule='adagrad',
                    loss='warp',
                    learning_rate=learning_rate,
                    item_alpha=item_alpha,
                    random_state=0)

    model.fit(interactions=train_interactions,
              item_features=item_features,
              epochs=3,
              verbose=True)

    test_precision = precision_at_k(model, test_interactions, k=10, item_features=item_features).mean()
    test_recall = recall_at_k(model, test_interactions, k=10, item_features=item_features).mean()
    test_auc = auc_score(model, test_interactions, item_features=item_features).mean()
    test_reciprocal = reciprocal_rank(model, test_interactions, item_features=item_features)
#     print(test_precision)
#     print(test_recall)
#     print(test_auc)
#     print(test_reciprocal)
    
    print("no_comp: {}, lrn_rate: {:.5f}, item_alpha: {:.5f}, precision: {:.5f}, recall: {:.5f}, reciprocal_rank: {}, auc_score: {:.5f}".format(
      no_components, learning_rate, item_alpha, test_precision, test_recall, test_reciprocal.shape, test_auc))
    # test_auc = auc_score(model, test, item_features=item_features).mean()
    output = -test_precision

    if np.abs(output+1) < 0.01 or output < -1.0:
        output = 0.0

    return output

In [ ]:
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=5, trials=trials)

best_params

In [ ]:
model = LightFM(no_components=40,
                learning_schedule='adagrad',
                loss='warp',
                learning_rate=0.011370592645615374,
                item_alpha=1.975734039413079e-05,
                random_state=42)

In [ ]:
%time model.fit_partial(train_interactions, item_features=item_features, epochs=10, verbose=True)

In [ ]:
print("Train precision: %.2f" % precision_at_k(model, test_interactions, k=10, item_features=item_features).mean())
print("Test precision: %.2f" % precision_at_k(model, test_interactions, k=10, item_features=item_features).mean())

In [ ]:
# test_recall = recall_at_k(model, test_interactions, k=10, item_features=item_features).mean()
test_auc = auc_score(model, test_interactions, item_features=item_features).mean()
test_reciprocal = reciprocal_rank(model, test_interactions, item_features=item_features).mean()

In [ ]:
print("AUC : %.5f  Reciprocal : %.5f" % (test_auc, test_reciprocal))

#### Evaluation

In [21]:
import pickle

In [22]:
# Load the saved model
with open('origin_user_rating_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [23]:
print("Train precision: %.5f" % precision_at_k(model, train_interactions, item_features=item_features, k=9).mean())
print("Test precision: %.5f" % precision_at_k(model, test_interactions, item_features=item_features, k=9).mean())

Train precision: 0.00601
Test precision: 0.00212


In [24]:
test_auc = auc_score(model, test_interactions, item_features=item_features).mean()
print(test_auc)

0.68399495


#### save

In [ ]:
# save the model to a file
with open('origin_user_rating_model.pkl', 'wb') as f:
    pickle.dump(model, f)